In [1]:
import os 

import numpy as np
import torch
import copy
import torch.nn.functional as F
from torch.nn import DataParallel

from torch import nn, optim
from resnet import resnet50

import pandas as pd
from PIL import ImageFile, Image
ImageFile.LOAD_TRUNCATED_IMAGES = True


import warnings
warnings.filterwarnings('ignore')

In [2]:
multi_gpus = False
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.device_count() > 1:
    multi_gpus = True
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
model = resnet50(196)

In [4]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [5]:
count_parameters(model)

23909636

In [6]:
import torchvision
from torchvision import datasets, transforms

def get_transform(random_crop=True):
    normalize = transforms.Normalize(
        mean=[x / 255.0 for x in [125.3, 123.0, 113.9]],
        std=[x / 255.0 for x in [63.0, 62.1, 66.7]]
        #[0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
        )
    transform = []
    transform.append(transforms.Resize(256))
    if random_crop:
        #transform.append(transforms.RandomRotation(30))
        transform.append(transforms.RandomResizedCrop(224))
        transform.append(transforms.RandomHorizontalFlip())
        transform.append(transforms.ColorJitter(hue=.05, saturation=.05),)
    else:
        transform.append(transforms.CenterCrop(224))
    transform.append(transforms.ToTensor())
    transform.append(normalize)
    return transforms.Compose(transform)

In [7]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils


class CarsDataset(Dataset):
    
    def __init__(self, csv_file, root_dir, transform=None):
        
        self.pd_csv = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.pd_csv)
    
    def __getitem__(self, index):
        
        if torch.is_tensor(index):
            index = index.tolist()
    
        img_name = os.path.join(self.root_dir, 
                                self.pd_csv.iloc[index, 1])
        image = Image.open(img_name).convert("RGB")
        
        if self.transform:
            image = self.transform(image)
            
        target = self.pd_csv.iloc[index, 0]

        return image, target

In [8]:
train_dataset = CarsDataset('./data/sfcars/train.csv', './data/sfcars/train/',
                                 transform=get_transform(random_crop=True))
test_dataset = CarsDataset('./data/sfcars/test.csv', './data/sfcars/test/',
                                 transform=get_transform(random_crop=False))

In [9]:
from torch.utils import data

tr_loader = data.DataLoader(dataset=train_dataset,
                            batch_size=256,
                            #sampler = RandomIdentitySampler(train_set, 4),
                            shuffle=True,
                            num_workers=16)

val_loader = data.DataLoader(dataset=test_dataset,
                             batch_size=256,
                             shuffle=False,                            
                             num_workers=16)

In [10]:
import apex
print("using apex synced BN")
model = apex.parallel.convert_syncbn_model(model)

using apex synced BN


In [11]:
optimizer = optim.SGD(model.parameters(), lr=1., momentum=0.9, weight_decay=5e-4, nesterov=True)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.1, steps_per_epoch=len(tr_loader)
                                                , epochs=100, pct_start=0.2)

In [12]:
from apex import amp, optimizers

model, optimizer = amp.initialize(model.cuda(), optimizer, opt_level='O3',keep_batchnorm_fp32=True)

Selected optimization level O3:  Pure FP16 training.
Defaults for this optimization level are:
enabled                : True
opt_level              : O3
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : False
master_weights         : False
loss_scale             : 1.0
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O3
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : True
master_weights         : False
loss_scale             : 1.0


In [13]:
criterion = nn.CrossEntropyLoss().cuda()


In [14]:
torch.cuda.synchronize()
model.train()
for _ in range(2):
    inputs, labels = next(iter(tr_loader))
    print(1)
    inputs = inputs.cuda(non_blocking=True)        
    labels = labels.cuda(non_blocking=True)    
    print(2)    
    logits = model(inputs)
    print(3)                       
    loss = criterion(logits, labels)                   
    print(4)                   
    with amp.scale_loss(loss, optimizer) as scaled_loss:
        scaled_loss.backward()
    print(5)                            
    model.zero_grad()
    print(10)                                
torch.cuda.synchronize()

1
2
3
4
5
10
1
2
3
4
5
10


In [15]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [16]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [17]:
import datetime
import time
high = 0.0
epoch_time = AverageMeter('Epoch', ':6.3f')
batch_time = AverageMeter('Batch', ':6.3f')
data_time = AverageMeter('Data', ':6.3f')
losses = AverageMeter('Loss', ':.5f')
learning_rates = AverageMeter('LearningRate', ':.5f')
top1 = AverageMeter('Acc@1', ':6.2f')
top5 = AverageMeter('Acc@5', ':6.2f')

for epoch in range(100):  # loop over the dataset multiple times
    time_ = datetime.datetime.now()    
    model.train()
    running_loss = 0.0
    running_corrects = 0
    total = 0
    progress = ProgressMeter(
        len(tr_loader),
        [batch_time, data_time, losses, top1, top5, learning_rates],
        prefix="Epoch: [{}]".format(epoch))
    
    end = time.time()    
    for i, (inputs, labels) in enumerate(tr_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        #print(inputs.shape)
        #print(labels.shape)
        data_time.update(time.time() - end)
        inputs = inputs.cuda(non_blocking=True)
        labels = labels.cuda(non_blocking=True)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        #_, preds = torch.max(outputs, 1)
        #loss.backward()
        with amp.scale_loss(loss, optimizer) as scaled_loss:
            scaled_loss.backward()
            
        optimizer.step()
        scheduler.step()
        # print statistics
        acc1, acc5 = accuracy(outputs, labels, topk=(1, 5))
        losses.update(loss.item(), inputs.size(0))
        learning_rates.update(scheduler.get_lr()[0])        
        top1.update(acc1[0], inputs.size(0))
        top5.update(acc5[0], inputs.size(0))

        
        batch_time.update(time.time() - end)
        if i % 100 == 99:    # print every 2000 mini-batches
            progress.display(i)
            #running_loss = 0.0
    progress.display(i)            
    elapsed = datetime.datetime.now() - time_
    print('{} elapsed for {}'.format(elapsed, epoch+1))

    
print('Finished Training')

Epoch: [0][31/32]	Batch 30.944 (22.541)	Data 30.646 (22.149)	Loss 5.33298 (5.35310)	Acc@1   0.00 (  0.58)	Acc@5   1.44 (  2.50)	LearningRate 0.00459 (0.00421)
0:00:31.077045 elapsed for 1
Epoch: [1][31/32]	Batch 30.966 (22.607)	Data 30.667 (22.207)	Loss 5.36528 (5.33306)	Acc@1   0.00 (  0.60)	Acc@5   1.44 (  2.85)	LearningRate 0.00636 (0.00481)
0:00:31.100765 elapsed for 2
Epoch: [2][31/32]	Batch 30.836 (22.497)	Data 30.538 (22.097)	Loss 5.33622 (5.32884)	Acc@1   1.44 (  0.71)	Acc@5   5.29 (  3.25)	LearningRate 0.00925 (0.00579)
0:00:30.957811 elapsed for 3
Epoch: [3][31/32]	Batch 29.484 (22.276)	Data 29.185 (21.872)	Loss 5.28964 (5.32946)	Acc@1   0.00 (  0.76)	Acc@5   2.88 (  3.51)	LearningRate 0.01319 (0.00714)
0:00:29.617346 elapsed for 4
Epoch: [4][31/32]	Batch 30.824 (22.239)	Data 30.526 (21.837)	Loss 5.24532 (5.31708)	Acc@1   1.44 (  0.89)	Acc@5   7.69 (  3.89)	LearningRate 0.01810 (0.00884)
0:00:30.953374 elapsed for 5
Epoch: [5][31/32]	Batch 29.777 (22.165)	Data 29.478 (21.764)

In [18]:
def classification_val(model, val_loader):
    correct = 0
    total = 0    
    
    model.eval()
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct/total

In [19]:
classification_val(model, val_loader)

0.7163288148240269

In [20]:
def val_retrieval(model, val_loader):
    feats = None
    data_ids = None

    model.eval()
    with torch.no_grad():
        for idx, (images, labels) in enumerate(val_loader):
            images = images.to(device)
            #labels = labels.to(device)

            feat = model(images, feature=True)
            feat = feat.detach().cpu().numpy()

            feat = feat/np.linalg.norm(feat, axis=1)[:, np.newaxis]

            if feats is None:
                feats = feat
            else:
                feats = np.append(feats, feat, axis=0)

            if data_ids is None:
                data_ids = labels
            else:
                data_ids = np.append(data_ids, labels, axis=0)

        score_matrix = feats.dot(feats.T)
        np.fill_diagonal(score_matrix, -np.inf)
        top1_reference_indices = np.argmax(score_matrix, axis=1)

        top1_reference_ids = [
            [data_ids[idx], data_ids[top1_reference_indices[idx]]] for idx in
            range(len(data_ids))]

    total_count = len(top1_reference_ids)
    correct = 0
    for ids in top1_reference_ids:
        if ids[0] == ids[1]:
            correct += 1        
    return correct/total_count

In [21]:
val_retrieval(model, val_loader)

0.6301455042905111

In [22]:
torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'loss': loss,    
    
}, './checkpoint/sfcar_resnet50_ep100.b0.pth')